# Motiviation

We already have a decision tree $F$ implemented, however it is impossible to learn all samples correctly in one pass, even with infinitely deep depth of the tree. How to improve without any modification to $F$?

The idea of an additive model comes into mind. Can we find any $h$ such that $y = F(x) + h(x)$, where $h$ focuses on the wrong predictions that $F$ makes. If found $h$ is unsatisfactory, we can just rinse and repeat by making new $F = F + h$ and find a new $h$. 

# Math

Say the goal is to minimize a loss function $J$, we can have the derivation as
$$J(y, F) = \sum_i^n L\big(y_i, F(x_i)\big)$$
where $L$ is a loss function comparing between ground truths and model's prediction.
Employing gradient descent is probably a good idea here. Therefore we have,
$$F_b(x_i) = F_{b-1}(x_i) - \eta \times \nabla L\big(y_i, F(x_i)\big)$$
where $F_b$ is our model at step $b$, $\eta$ is the learning rate.



## Regression

When it is regression, we are considering $L$ to be MSE:
$$L\big(y_i, F(x_i)\big) = \frac{1}{2} \big( y_i - F(x_i) \big)^2$$
Taking the gradient, we would have
$$\frac{ \partial L\big(y_i, F(x_i)\big) }{ \partial F(x_i) } = \frac{ \partial \frac{1}{2} \big( y_i - F(x_i) \big)^2 }{ \partial F(x_i) } = F(x_i) - y_i$$

Substitute back in, we would have
$$\begin{align}
F_b(x_i) &= F_{b-1}(x_i) + h(x_i) \nonumber \\
         &= F_{b-1}(x_i) + y_i - F_{b-1}(x_i) \nonumber \\
         &= F_{b-1}(x_i) - \frac{ \partial L\big(y_i, F_{b-1}(x_i)\big) }{ \partial F_{b-1}(x_i) }
         \nonumber \\
\end{align}$$

Hence, to obtain a $F_{b-1}(x_i)$ that is $F_{b-1}(x_i) = F_{b}(x_i) = y$, we are essentially minimizing $- \eta \times \nabla L\big(y_i, F(x_i)\big)$. Rearrange the equation, we would have $F_b(x_i) - F_{b-1}(x_i) = - \frac{ \partial L\big(y_i, F_{b-1}(x_i)\big) }{ \partial F_{b-1}(x_i) }$, which essentially is $y_i - F_{b-1}(x_i) = - \frac{ \partial L\big(y_i, F_{b-1}(x_i)\big) }{ \partial F_{b-1}(x_i) }$. Now in code implementation, all we have to do is to minimize $y_i - F_{b-1}(x_i)$, which we can refer as **residual**.

## Classification

When it is classification, we are considering $L$ to be cross entropy loss:
$$L\big(y_i, F(x_i)\big) = -\sum_k ^ K y_k(x_i) \log p_k(x_i)$$
$y_k(x_i)$ will be a one hot vector of length $k$, with $1$ as the $i$th entry where $y_i = k$ and $0$ everywhere else, and $p_k(x_i)$ would be the predicted probability of sample $x_i$ belonging to class $k$ by our model.

To obtain $p$, we can perform softmax transformation on $o$ which is the raw output (logits) from our model. Referencing how we handle this in logistic regression, we can know that
$$
\frac{\partial p_i}{\partial o_j} = \frac{\partial \frac{e^{o_i}}{\sum_{k=1}^{N}e^{o_k}}}{\partial o_j}
$$
knowing that $f'(x) = \frac{g'(x)h(x) - h'(x)g(x)}{[h(x)]^2}$ and $$\begin{align*}
g &= e^{o_i} \nonumber \\
h &= \sum_{k=1}^{K}e^{o_k} \nonumber
\end{align*}$$
when $i = j$ we have
$$\begin{align*}
\frac{\partial \frac{e^{o_i}}{\sum_{k = 1}^{N} e^{o_k}}}{\partial o_j} 
&= \frac{e^{o_i}\Sigma-e^{o_j}e^{o_i}}{\Sigma^2} \nonumber \\
&= \frac{e^{o_i}}{\Sigma}\frac{\Sigma - e^{o_j}}{\Sigma} \nonumber \\
&= p_i(1 - p_j) \nonumber \\
&= p_i(1 - p_i) \nonumber
\end{align*}$$,
and when $ i \neq j$, we have
$$\begin{align*}
\frac{\partial \frac{e^{o_i}}{\sum_{k = 1}^{N} e^{o_k}}}{\partial o_j} 
&= \frac{0-e^{o_j}e^{o_i}}{\Sigma^2} \nonumber \\
&= -\frac{e^{o_j}}{\Sigma}\frac{e^{o_i}}{\Sigma} \nonumber \\
&= -p_j p_i \nonumber \\
&= -p_i p_j \nonumber
\end{align*}$$

Plug them all back in, we have
$$\begin{align}
\frac{\partial L}{\partial o_i} 
&= -\sum_k y_k\frac{\partial \log p_k}{\partial o_i} \nonumber \\
&= -\sum_k y_k\frac{1}{p_k}\frac{\partial p_k}{\partial o_i} \nonumber \\
&= -y_i(1-p_i) - \sum_{k \neq i}y_k\frac{1}{p_k}(-p_kp_i) \nonumber \\
&= -y_i(1 - p_i) + \sum_{k \neq i}y_k(p_i) \nonumber \\
&= -y_i + y_i p_i + \sum_{k \neq i}y_k(p_i) \nonumber \\
&= p_i\left(\sum_ky_k\right) - y_i \nonumber \\
&= p_i - y_i \nonumber
\end{align}$$

The negative gradient is the **residual** in the case of classification, which is $y_i - F_{b-1}(x_i)$